In [17]:
import tensorflow as tf
import numpy as np
import json
import os

def zapisz_json(co,gdzie):
    f=open(gdzie,'w')
    f.write(json.dumps(co))

def wczytaj_json(skad):
    f=open(skad,'r')
    return json.loads(f.read())

# The following functions can be used to convert a value to a type compatible
# with tf.Example. values to lista takich rzeczy.

def _bytes_feature(values):
  """Returns a bytes_list from a string / byte."""
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=values))

def _float_feature(values):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=values))

def _int64_feature(values):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=values))

#nazwa tworzonego, slownik typu nazwa -> rozmiar, c to typu nazwa -> lista_wartosci
# zwraca writer sluzacy do pisania do tego czegos
def tworz_folder_na_podstawie_przykladowego(nazwa_folderu, n, c):
    assert not os.path.isdir(nazwa_folderu)
    os.mkdir(nazwa_folderu)

    lista_numerycznych_typow = []
    rozmiar_dotychczasowego = 0
    for k in n.keys():
        lista_numerycznych_typow.append((k, rozmiar_dotychczasowego, rozmiar_dotychczasowego + n[k]))
        rozmiar_dotychczasowego += n[k]
    zapisz_json(lista_numerycznych_typow, nazwa_folderu + "/n")
    
    lista_kategorycznch_typow = []
    for k in c.keys():
        lista_kategorycznch_typow.append((k, c[k]))
    zapisz_json(lista_kategorycznch_typow, nazwa_folderu + "/c")
    writer = tf.python_io.TFRecordWriter(nazwa_folderu + '/dane')
    return lista_numerycznych_typow, lista_kategorycznch_typow, writer

# n to slownik nazwa -> lista lub numpy tablica shapu (k,), c to nazwa -> wartosc, l to 0 lub 1
#pamietaj by po wszystkim zrobic writer.close()
def zapisz_jeden_przyklad(n, c, l, lnt, lkt, writer):
    slownik_danych = {}
    slownik_danych['l'] = _int64_feature([l])
    
    lista_numerycznych = []
    for i in range(len(lnt)):
        rozwazany = lnt[i]
        lista_numerycznych += n[rozwazany[0]]
    slownik_danych['n']= _float_feature(lista_numerycznych)

    lista_kategorycznch= []
    for i in range(len(lkt)):
        rozwazany = lkt[i]
        lista_kategorycznch += [c[rozwazany[0]] ]
    slownik_danych['c']= _int64_feature(lista_kategorycznch)
    

    tr_ex = tf.train.Example(features= tf.train.Features(
        feature = slownik_danych))
    writer.write(tr_ex.SerializeToString())

def tworz_slownik_typow_potrzebny_do_parsowania(nazwa_folderu):
    lista_numerycznych = wczytaj_json(nazwa_folderu + '/n')
    laczna_liczba = lista_numerycznych[len(lista_numerycznych) - 1][2]
    lista_kategorycznych = wczytaj_json(nazwa_folderu + '/c')

    read_features= {}
    read_features['n'] = tf.FixedLenFeature([laczna_liczba], dtype=tf.float32)
    read_features['c'] = tf.FixedLenFeature([len(lista_kategorycznych)], dtype=tf.int64)
    read_features['l'] = tf.FixedLenFeature([1], dtype=tf.int64)
    return read_features

def wyrwij_label(slownik):
    l= slownik.pop('l')
    return slownik, l

def odczytaj_dataset(nazwa_folderu):
    dataset = tf.data.TFRecordDataset(nazwa_folderu + '/dane')
    return dataset

def parsuj_po_jednym(nazwa_folderu):
    read_features = tworz_slownik_typow_potrzebny_do_parsowania(nazwa_folderu)
    def parsuj(serialized_example):
        return tf.parse_single_example(serialized=serialized_example,
                                        features=read_features)
    dataset = odczytaj_dataset(nazwa_folderu)
    dataset = dataset.map(parsuj)
    return dataset

def parsuj_i_batchuj(nazwa_folderu, batch_size):
    read_features = tworz_slownik_typow_potrzebny_do_parsowania(nazwa_folderu)
    def parsuj(serialized_examples):
         return tf.parse_example(serialized=serialized_examples,
                                        features=read_features)
    dataset = odczytaj_dataset(nazwa_folderu)
    dataset= dataset.batch(batch_size)
    dataset = dataset.map(parsuj)
    return dataset


    
    





In [2]:
lnt, lkt, writer = tworz_folder_na_podstawie_przykladowego('pierwszy', {'a': 2, 'b': 3}, {'c': [1,2,5]})

In [3]:
lkt

[('c', [1, 2, 5])]

In [6]:
for i in range(1000):
    zapisz_jeden_przyklad({'a':[5,6], 'b':[8,9,11]}, {'c':5}, 0, lnt, lkt, writer)

In [7]:
zapisz_jeden_przyklad({'a':[50,60], 'b':[80,90,110]}, {'c':5}, 1, lnt, lkt, writer)

In [8]:
writer.close()

In [18]:
dataset = parsuj_i_batchuj('pierwszy',3)
costam = dataset.make_one_shot_iterator()

In [19]:
sess = tf.InteractiveSession()

/home/pawel/anaconda3/envs/root_tensorflow/lib/python3.6/site-packages/tensorflow/python/client/session.py:1711: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [20]:

sess.run(costam.get_next())

{'c': array([[5],
        [5],
        [5]]), 'l': array([[0],
        [1],
        [0]]), 'n': array([[  5.,   6.,   8.,   9.,  11.],
        [ 50.,  60.,  80.,  90., 110.],
        [  5.,   6.,   8.,   9.,  11.]], dtype=float32)}